In [1]:


from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
tf.keras.backend.floatx()
import pandas as pd
#CSV_COLUMN_NAMES =['# of Peers','Endorsing','MaxDelay','MinDelay',
                   #'AvgDelay','TPS','IsTrustable']
SPECIES = ['Trustable','Non-trustable']
dataset = pd.read_csv('/home/inlab/Downloads/data extraction_0125_v1.csv')
#define some constants 
#train = pd.read_csv("/home/inlab/Downloads/trainINLAB.csv", names=CSV_COLUMN_NAMES, header=0)
#test = pd.read_csv("/home/inlab/Downloads/evalINLAB.csv", names=CSV_COLUMN_NAMES, header=0)
#use keras (a module inside of TensorFlow) to grab datasets and read them into a pandas dataframe







In [2]:
dataset.head()

,NoPeers,Endorsing,MaxDelay,MinDelay,AvgDelay,IsTrustable
0,20,20,98,28,67.85,1
1,20,20,99,23,58.90,0
2,20,20,89,29,67.95,1
3,20,20,99,21,60.00,1
4,20,20,98,25,62.45,0


In [ ]:
test.head()

In [3]:
dataset.corr('pearson')

,NoPeers,Endorsing,MaxDelay,MinDelay,AvgDelay,IsTrustable
NoPeers,1.000000,1.000000,0.185710,-0.191541,0.845770,-0.234630
Endorsing,1.000000,1.000000,0.185710,-0.191541,0.845770,-0.234630
MaxDelay,0.185710,0.185710,1.000000,0.090402,0.284153,-0.132432
MinDelay,-0.191541,-0.191541,0.090402,1.000000,0.051230,-0.051274
AvgDelay,0.845770,0.845770,0.284153,0.051230,1.000000,-0.246264
IsTrustable,-0.234630,-0.234630,-0.132432,-0.051274,-0.246264,1.000000


In [ ]:
train_y = train.pop('HandoverResult')
test_y = test.pop('HandoverResult')
test.head() # the species column is now gone

In [4]:
X = dataset.drop(['IsTrustable'], axis=1)
X.head()

,NoPeers,Endorsing,MaxDelay,MinDelay,AvgDelay
0,20,20,98,28,67.85
1,20,20,99,23,58.90
2,20,20,89,29,67.95
3,20,20,99,21,60.00
4,20,20,98,25,62.45


In [5]:
y = dataset['IsTrustable']
y.head()

0    1
1    0
2    1
3    1
4    0
Name: IsTrustable, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=20)
train, test, train_y, test_y = train_test_split(X,y,test_size=0.2,random_state=20)

In [7]:
print('The size of the training "X" (input features) is', train.shape)
print('\n')
print('The size of our testing "X" (input features) is', test.shape)
print('\n')
print('The size of the training "y" (output features) is', train_y.shape)
print('\n')
print('The size of the training "y" (output features) is', test_y.shape)

The size of the training "X" (input features) is (140, 5)


The size of our testing "X" (input features) is (36, 5)


The size of the training "y" (output features) is (140,)


The size of the training "y" (output features) is (36,)


In [ ]:
test.shape  # 179 entires with 5 features

In [ ]:
train.shape

In [ ]:
train.describe()

In [8]:
def input_fn(features, labels, training=True, batch_size=128):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    # Shuffle and repeat if in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)



[NumericColumn(key='NoPeers', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Endorsing', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MaxDelay', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MinDelay', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='AvgDelay', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [9]:
# Build a DNN with 3 hidden layers with 20, 10 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # 3 hidden layers of 20, 10 and 10 nodes respectively.
    hidden_units=[20,10,10],
    # The model must choose between 2 classes.
    n_classes=2)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpu3k3l506', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
train_result = classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=50000)
# We include a lambda to avoid creating an inner function previously


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

INFO:tensorflow:loss = 0.64765626, step = 5700 (0.168 sec)
INFO:tensorflow:global_step/sec: 551.61
INFO:tensorflow:loss = 0.6302812, step = 5800 (0.181 sec)
INFO:tensorflow:global_step/sec: 588.498
INFO:tensorflow:loss = 0.65871227, step = 5900 (0.170 sec)
INFO:tensorflow:global_step/sec: 641.321
INFO:tensorflow:loss = 0.63343906, step = 6000 (0.156 sec)
INFO:tensorflow:global_step/sec: 515.261
INFO:tensorflow:loss = 0.6685807, step = 6100 (0.194 sec)
INFO:tensorflow:global_step/sec: 534.9
INFO:tensorflow:loss = 0.6643375, step = 6200 (0.189 sec)
INFO:tensorflow:global_step/sec: 319.39
INFO:tensorflow:loss = 0.6509441, step = 6300 (0.311 sec)
INFO:tensorflow:global_step/sec: 694.567
INFO:tensorflow:loss = 0.6392479, step = 6400 (0.144 sec)
INFO:tensorflow:global_step/sec: 754.935
INFO:tensorflow:loss = 0.6632702, step = 6500 (0.132 sec)
INFO:tensorflow:global_step/sec: 708.569
INFO:tensorflow:loss = 0.66940206, step = 6600 (0.141 sec)
INFO:tensorflow:global_step/sec: 661.076
INFO:tenso

INFO:tensorflow:global_step/sec: 556.033
INFO:tensorflow:loss = 0.65443873, step = 14000 (0.180 sec)
INFO:tensorflow:global_step/sec: 699.431
INFO:tensorflow:loss = 0.65844774, step = 14100 (0.143 sec)
INFO:tensorflow:global_step/sec: 653.113
INFO:tensorflow:loss = 0.64185923, step = 14200 (0.153 sec)
INFO:tensorflow:global_step/sec: 644.145
INFO:tensorflow:loss = 0.6765653, step = 14300 (0.155 sec)
INFO:tensorflow:global_step/sec: 554.075
INFO:tensorflow:loss = 0.65094787, step = 14400 (0.180 sec)
INFO:tensorflow:global_step/sec: 710.35
INFO:tensorflow:loss = 0.6641076, step = 14500 (0.141 sec)
INFO:tensorflow:global_step/sec: 575.282
INFO:tensorflow:loss = 0.685634, step = 14600 (0.174 sec)
INFO:tensorflow:global_step/sec: 462.044
INFO:tensorflow:loss = 0.6541639, step = 14700 (0.216 sec)
INFO:tensorflow:global_step/sec: 493.64
INFO:tensorflow:loss = 0.6885031, step = 14800 (0.204 sec)
INFO:tensorflow:global_step/sec: 624.94
INFO:tensorflow:loss = 0.674189, step = 14900 (0.158 sec)
I

INFO:tensorflow:global_step/sec: 374.192
INFO:tensorflow:loss = 0.64513737, step = 22200 (0.265 sec)
INFO:tensorflow:global_step/sec: 481.566
INFO:tensorflow:loss = 0.6550464, step = 22300 (0.208 sec)
INFO:tensorflow:global_step/sec: 644.334
INFO:tensorflow:loss = 0.6576784, step = 22400 (0.155 sec)
INFO:tensorflow:global_step/sec: 340.985
INFO:tensorflow:loss = 0.6739496, step = 22500 (0.295 sec)
INFO:tensorflow:global_step/sec: 571.546
INFO:tensorflow:loss = 0.6428294, step = 22600 (0.173 sec)
INFO:tensorflow:global_step/sec: 632.117
INFO:tensorflow:loss = 0.6562842, step = 22700 (0.158 sec)
INFO:tensorflow:global_step/sec: 439.231
INFO:tensorflow:loss = 0.6560873, step = 22800 (0.230 sec)
INFO:tensorflow:global_step/sec: 623.428
INFO:tensorflow:loss = 0.64486754, step = 22900 (0.161 sec)
INFO:tensorflow:global_step/sec: 419.555
INFO:tensorflow:loss = 0.65234727, step = 23000 (0.236 sec)
INFO:tensorflow:global_step/sec: 403.907
INFO:tensorflow:loss = 0.65137696, step = 23100 (0.247 s

INFO:tensorflow:global_step/sec: 690.771
INFO:tensorflow:loss = 0.65381706, step = 30400 (0.145 sec)
INFO:tensorflow:global_step/sec: 593.063
INFO:tensorflow:loss = 0.6755514, step = 30500 (0.169 sec)
INFO:tensorflow:global_step/sec: 697.499
INFO:tensorflow:loss = 0.6608361, step = 30600 (0.143 sec)
INFO:tensorflow:global_step/sec: 684.721
INFO:tensorflow:loss = 0.673226, step = 30700 (0.146 sec)
INFO:tensorflow:global_step/sec: 477.093
INFO:tensorflow:loss = 0.6520608, step = 30800 (0.210 sec)
INFO:tensorflow:global_step/sec: 434.269
INFO:tensorflow:loss = 0.6463355, step = 30900 (0.230 sec)
INFO:tensorflow:global_step/sec: 578.155
INFO:tensorflow:loss = 0.62579966, step = 31000 (0.174 sec)
INFO:tensorflow:global_step/sec: 590.16
INFO:tensorflow:loss = 0.6575792, step = 31100 (0.167 sec)
INFO:tensorflow:global_step/sec: 542.823
INFO:tensorflow:loss = 0.6592363, step = 31200 (0.184 sec)
INFO:tensorflow:global_step/sec: 719.448
INFO:tensorflow:loss = 0.64792687, step = 31300 (0.139 sec)

INFO:tensorflow:global_step/sec: 297.786
INFO:tensorflow:loss = 0.6332139, step = 38600 (0.336 sec)
INFO:tensorflow:global_step/sec: 633.067
INFO:tensorflow:loss = 0.6623295, step = 38700 (0.158 sec)
INFO:tensorflow:global_step/sec: 541.488
INFO:tensorflow:loss = 0.6437927, step = 38800 (0.185 sec)
INFO:tensorflow:global_step/sec: 571.057
INFO:tensorflow:loss = 0.65763897, step = 38900 (0.175 sec)
INFO:tensorflow:global_step/sec: 600.864
INFO:tensorflow:loss = 0.6548879, step = 39000 (0.166 sec)
INFO:tensorflow:global_step/sec: 435.262
INFO:tensorflow:loss = 0.6529325, step = 39100 (0.230 sec)
INFO:tensorflow:global_step/sec: 638.387
INFO:tensorflow:loss = 0.6563596, step = 39200 (0.157 sec)
INFO:tensorflow:global_step/sec: 606.024
INFO:tensorflow:loss = 0.6544086, step = 39300 (0.165 sec)
INFO:tensorflow:global_step/sec: 400.075
INFO:tensorflow:loss = 0.67248285, step = 39400 (0.250 sec)
INFO:tensorflow:global_step/sec: 668.649
INFO:tensorflow:loss = 0.6328253, step = 39500 (0.150 sec

INFO:tensorflow:global_step/sec: 503.624
INFO:tensorflow:loss = 0.66720897, step = 46500 (0.199 sec)
INFO:tensorflow:global_step/sec: 622.131
INFO:tensorflow:loss = 0.6507553, step = 46600 (0.161 sec)
INFO:tensorflow:global_step/sec: 369.747
INFO:tensorflow:loss = 0.6750456, step = 46700 (0.270 sec)
INFO:tensorflow:global_step/sec: 579.731
INFO:tensorflow:loss = 0.6484082, step = 46800 (0.173 sec)
INFO:tensorflow:global_step/sec: 550.115
INFO:tensorflow:loss = 0.6477519, step = 46900 (0.182 sec)
INFO:tensorflow:global_step/sec: 444.964
INFO:tensorflow:loss = 0.633481, step = 47000 (0.225 sec)
INFO:tensorflow:global_step/sec: 511.51
INFO:tensorflow:loss = 0.66042435, step = 47100 (0.196 sec)
INFO:tensorflow:global_step/sec: 580.202
INFO:tensorflow:loss = 0.6672193, step = 47200 (0.172 sec)
INFO:tensorflow:global_step/sec: 547.061
INFO:tensorflow:loss = 0.6465075, step = 47300 (0.181 sec)
INFO:tensorflow:global_step/sec: 626.12
INFO:tensorflow:loss = 0.66054624, step = 47400 (0.160 sec)


In [11]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-27T17:21:38Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpu3k3l506/model.ckpt-50000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.67748s
INFO:tensorflow:Finished evaluation at 2021-01-27-17:21:39
INFO:tensorflow:Saving dict for global step 50000: accuracy = 0.6666667, accur

In [12]:
#Improving the model
#first is to normalize the training data
train_min = train.min()
train_min

NoPeers      15.0
Endorsing    15.0
MaxDelay     78.0
MinDelay     20.0
AvgDelay     37.6
dtype: float64

In [13]:
train_max = train.max()
train_max

NoPeers      20.0
Endorsing    20.0
MaxDelay     99.0
MinDelay     45.0
AvgDelay     74.8
dtype: float64

In [14]:
train_range = (train_max-train_min)
train_range

NoPeers       5.0
Endorsing     5.0
MaxDelay     21.0
MinDelay     25.0
AvgDelay     37.2
dtype: float64

In [15]:
train_scaled = (train - train_min)/(train_range)
train_scaled.head()

,NoPeers,Endorsing,MaxDelay,MinDelay,AvgDelay
1,1.0,1.0,1.000000,0.12,0.572581
67,1.0,1.0,0.761905,0.20,0.711022
172,0.0,0.0,0.809524,0.96,0.404570
64,1.0,1.0,0.952381,0.40,0.931452
20,1.0,1.0,0.952381,0.20,0.955645


In [16]:
#Train the SVM model with the scaled(Normalized)datasets
test_min = test.min()
test_range = (test - test_min).max()
test_scaled = (test - test_min)/test_range

In [17]:
train_result = classifier.train(
    input_fn=lambda: input_fn(train_scaled, train_y, training=True),
    steps=5000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpu3k3l506/model.ckpt-50000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 50000...
INFO:tensorflow:Saving checkpoints for 50000 i

In [18]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test_scaled, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-27T17:22:15Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpu3k3l506/model.ckpt-55000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.67870s
INFO:tensorflow:Finished evaluation at 2021-01-27-17:22:15
INFO:tensorflow:Saving dict for global step 55000: accuracy = 0.6666667, accur

In [ ]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['CurrentAPSignal','NeighboringAPSignal','Mobility','BlockPropagationDelay',
                   'BlockReceivingTime']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
    val = input(feature + ": ")
    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))
